## Minimax

In [ ]:
import os.path
css = ""
if os.path.isfile("style.css"):
    from IPython.core.display import HTML
    with open("style.css", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb

`memoize(f)` takes a function and checks if the function was already called with the same two main arguments and returns the saved return value. If it is not yet in the Cache, the arguments and it's return values will be saved to the Cache

Die Funktion `memoize(f)` nimmt eine Funktion `f` und überprüft ob diese bereits mit den beiden gleichen Haupt-Parametern aufgerufen wurde und liefert den gespeicherten Antwortwert. Wenn die Funktion bisher nicht aufgerufen wurde, werden die Parameter und die Antwortwerte in den Cache gespeichert

In [ ]:
Cache = {}

def memoize(f):
    global Cache

    def f_memoized(*args):
        key = (args[0], args[1])
        if key in Cache:
            return Cache[key]
        result = f(*args)
        Cache[key] = result
        return result

    return f_memoized

Die `altScore(s, p)` Funktion dient zu einer hilfsweisen Berechnung von Siegwahrscheinlichkeiten anhand der Gegebenheiten. 
Je höher der Wert ist, umso besser stehen die Siegwahrscheinlichkeiten von `p`.

Vorhandene Steine zählen einfach in die Berechnung, vorhandene Mühlen vierfach und mögliche Mühlen doppelt.

Der schlechteste mögliche Wert ist -0.9, der beste 0.9.

In [ ]:
def altScore(s, p):
    op = opponent(p)
    (_, board) = s
    # finding possible mills in next states
    pPossMills = sum(len(findMills(nB, p)) for (_, nB) in nextStates(s, p))
    opPossMills = sum(len(findMills(nB, op)) for (_, nB) in nextStates(s, op))
    # adding all score parameters
    pScore = countStones(s, p) + len(findMills(board, p)) * 4 + pPossMills * 2
    opScore = countStones(s, op) + len(findMills(board, op)) * 4 + opPossMills * 2
    # subtracting opponent score from own players score
    result = pScore - opScore
    # limeting the score to not overwrite real value score
    if result > 90:
        result = 90
    if result < -90:
        result = -90
    return result / 100

The `value(s, p, count)` function takes the current state and the player and returns the value (see `utility(s, p)`) of the winner if every move is optimal
the `count` variable is used to limit the recursion depth.

Die `value(s, p, count)` Funktion nimmt einen Spielzustand und einen Spieler und liefert den Wert (siehe `utility(s, p)`) des Gewinners wenn dieser alle Züge optimal ausführt.
Die Variable `count` wird dafür genutzt die Rekursionstiefe zu bestimmen.

In [ ]:
@memoize
def value(s, p, count):
    count += 1
    if finished(s, p):
        return utility(s, p)
    if count > 3:
        return altScore(s, p)
    return max([-value(ns, opponent(p), count) for ns in nextStates(s, p)])

`bestMove(s, p)` calculates the best move to win the game for the given state and player

Die Funktion `bestMove(s, p)` berechnet den günstigsten Zug für einen Spieler um das Spiel zu gewinnen, mit dem gegeben Ausgangszustand

In [ ]:
def bestMove(s, p):
    # Clear @memoize cache
    global Cache
    Cache = {}

    ns = nextStates(s, p)
    bestValue = value(s, p, 0)
    bestMoves = [s for s in ns if -value(s, opponent(p), 0) == bestValue]
    return bestValue, bestMoves[0]